In [92]:
%matplotlib inline
import pandas as pd
import pylab
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sci
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

import random
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.spatial.distance as ssd
from sklearn.ensemble import RandomForestClassifier
import sklearn
from time import time
import gc
from sklearn.neural_network import MLPClassifier

In [2]:
def mag(x):
    if x>0:
        return -2.5*np.log(x)
    else:
        return 10

In [3]:
def dist(x):
    if x>0:
        return 44+3*np.log(x)
    else:
        return 0

In [42]:
test_meta=pd.read_csv("/modules/cs342/Assignment2/test_set_metadata.csv")
object_id=test_meta['object_id']

In [4]:
test=pd.read_csv('/dcs/17/u1403100/Downloads/A2/test_final.csv')
train=pd.read_csv('/dcs/17/u1403100/Downloads/A2/train_final.csv')


In [5]:
"""
test.drop(list(test.filter(regex = 'object')), axis = 1, inplace = True)
test.drop(['last_row'],axis=1,inplace=True)
test.rename(columns={'duration': 'detection_length'}, inplace=True)
"""

"\ntest.drop(list(test.filter(regex = 'object')), axis = 1, inplace = True)\ntest.drop(['last_row'],axis=1,inplace=True)\ntest.rename(columns={'duration': 'detection_length'}, inplace=True)\n"

In [6]:
"""test['distmod']=test['hostgal_photoz'].apply(dist)
for i in range(6):
    test[str(i)+'_band_abs_mag']=test[str(i)+'_band_Max'].apply(mag)-test['distmod']

test.drop('hostgal_specz',axis=1,inplace=True)"""

"test['distmod']=test['hostgal_photoz'].apply(dist)\nfor i in range(6):\n    test[str(i)+'_band_abs_mag']=test[str(i)+'_band_Max'].apply(mag)-test['distmod']\n\ntest.drop('hostgal_specz',axis=1,inplace=True)"

In [7]:
"""train['distmod']=train['hostgal_photoz'].apply(dist)
for i in range(6):
    train[str(i)+'_band_abs_mag']=train[str(i)+'_band_Max'].apply(mag)-train['distmod']

train.drop('hostgal_specz',axis=1,inplace=True)
test['0_band_Median']=test['0_band_Median'].fillna(0)"""

"train['distmod']=train['hostgal_photoz'].apply(dist)\nfor i in range(6):\n    train[str(i)+'_band_abs_mag']=train[str(i)+'_band_Max'].apply(mag)-train['distmod']\n\ntrain.drop('hostgal_specz',axis=1,inplace=True)\ntest['0_band_Median']=test['0_band_Median'].fillna(0)"

In [8]:
"""test.fillna(0,inplace=True)
test=test.replace([np.inf,-np.inf], 0)"""

'test.fillna(0,inplace=True)\ntest=test.replace([np.inf,-np.inf], 0)'

In [15]:

train_x=train.drop(['target','object_id'],axis=1)
train_y=train['target'].fillna(99)



(31392, 46)

In [29]:
rfc=RandomForestClassifier(n_estimators=90, min_samples_leaf=10,n_jobs=1)
rfc.fit(train_x,train_y)
print(rfc.classes_)



[ 6 15 16 42 52 53 62 64 65 67 88 90 92 95]


In [30]:
pred=rfc.predict_proba(test.loc[:1800000,:])


In [36]:
pred2=rfc.predict_proba(test.loc[1800001:,:])

In [25]:
test.shape

(3492890, 46)

In [38]:
pred.shape

(1800001, 14)

In [39]:
pred2.shape

(1692889, 14)

In [40]:
pred_final=pd.concat([pred,pred2],axis=0)

TypeError: cannot concatenate a non-NDFrame object

In [48]:
    dataset = pd.DataFrame({
                        'Class_6':pred[:,0],
                        'Class_15':pred[:,1],
                        'Class_16':pred[:,2],
                        'Class_42':pred[:,3],
                        'Class_52':pred[:,4],
                        'Class_53':pred[:,5],
                        'Class_62':pred[:,6],
                        'Class_64':pred[:,7],
                        'Class_65':pred[:,8],
                        'Class_67':pred[:,9],
                        'Class_88':pred[:,10],
                        'Class_90':pred[:,11],
                        'Class_92':pred[:,12],
                        'Class_95':pred[:,13]                              
                       })
   

In [49]:
    dataset2 = pd.DataFrame({
                        'Class_6':pred2[:,0],
                        'Class_15':pred2[:,1],
                        'Class_16':pred2[:,2],
                        'Class_42':pred2[:,3],
                        'Class_52':pred2[:,4],
                        'Class_53':pred2[:,5],
                        'Class_62':pred2[:,6],
                        'Class_64':pred2[:,7],
                        'Class_65':pred2[:,8],
                        'Class_67':pred2[:,9],
                        'Class_88':pred2[:,10],
                        'Class_90':pred2[:,11],
                        'Class_92':pred2[:,12],
                        'Class_95':pred2[:,13]                              
                       })


In [81]:
output=pd.concat([dataset,dataset2],axis=0)
output.index = np.arange(0, len(output))

In [90]:
output['object_id']=test_meta['object_id']
output['class_99']=0.05

In [91]:
output.to_csv('/dcs/17/u1403100/Downloads/A2/task5_1_2.csv',index=False)

In [78]:
output2=output.reset_index

In [62]:
test_meta.shape

(3492890, 11)

In [94]:
mlp = MLPClassifier(max_iter=100,alpha= 0.01, activation='tanh',learning_rate='adaptive',hidden_layer_sizes=(30,))
mlp.fit(train_x,train_y)
print(mlp.classes_)

[ 6 15 16 42 52 53 62 64 65 67 88 90 92 95]


In [95]:
pred=mlp.predict_proba(test)

In [97]:
    dataset = pd.DataFrame({
                        'Class_6':pred[:,0],
                        'Class_15':pred[:,1],
                        'Class_16':pred[:,2],
                        'Class_42':pred[:,3],
                        'Class_52':pred[:,4],
                        'Class_53':pred[:,5],
                        'Class_62':pred[:,6],
                        'Class_64':pred[:,7],
                        'Class_65':pred[:,8],
                        'Class_67':pred[:,9],
                        'Class_88':pred[:,10],
                        'Class_90':pred[:,11],
                        'Class_92':pred[:,12],
                        'Class_95':pred[:,13]                              
                       })
output=dataset
output['object_id']=test_meta['object_id']   

In [99]:
output['class_99']=0.08
output.to_csv('/dcs/17/u1403100/Downloads/A2/task5_2.csv',index=False)